In [2]:
import pandas as pd
import gzip
import json


def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield json.loads(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('..\data\meta_Gift_Cards.json.gz')


In [17]:
from nltk.stem import SnowballStemmer  
from nltk.tokenize import word_tokenize

snowball_stemmer = SnowballStemmer("english")  

In [69]:
import re
def get_info(df,col_name):
    info = {'index': {}, 'wordcnt': {}, 'doclen': {}}
    for index, row in df.iterrows():
        text = str(row[col_name])
        raw_str = re.sub('[^\w ]', '', text)
        tokens = word_tokenize(raw_str)
        terms = [snowball_stemmer.stem(token) for token in tokens]
        id = row['asin']
        for term in terms:
            if term not in info['wordcnt']:
                info['wordcnt'][term] = 0
            info['wordcnt'][term] += 1
            
        doclen = len(terms)
        terms_uni = set(terms)
        tokens_uni = set(tokens)
        doclen_uni = len(terms_uni)
        
        inverted_index = {}
        for term in terms_uni:
            if term not in info['index']:
                info['index'][term] = []
            info['index'][term].append(id)
        for token in tokens_uni:
            if token not in info['index']:
                info['index'][token] = []
            info['index'][term].append(id)   
            
        info['doclen'][id] = [doclen, doclen_uni]
    return info

title_info = get_info(df,'title')
feature_info = get_info(df,'feature')
description_info = get_info(df,'description')


In [71]:
def info_dump(info,path):
    with open(path,'w') as f:
        json.dump(info,f)

info_dump(title_info['index'],'../data/inverted_index/title_index.json')
info_dump(feature_info['index'], '../data/inverted_index/feature_index.json')
info_dump(description_info['index'], '../data/inverted_index/description_index.json')

In [73]:
test=[1,2,3]
json.dump(test,'../data/inverted_index/test.json')

AttributeError: 'str' object has no attribute 'write'